# 1. Problem Statement

In this notebook, we aim to analyze air quality and global air pollution data.

# 2. Data Loading and Description

We will load the datasets and provide an overview of each.

In [ ]:

import pandas as pd

# Load the datasets
air_quality_data = pd.read_csv('AirQuality.csv')
global_air_pollution_data = pd.read_csv('global air pollution dataset.csv')

# Display the first few rows of both datasets
air_quality_data.head(), global_air_pollution_data.head()


# 3. Naive Submission 1

This is a simple initial submission with basic predictions.

# 4. Machine Learning Pipeline

We will describe the steps followed in creating the machine learning model.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

# Reload the data with the correct delimiter
air_quality_data = pd.read_csv('AirQuality.csv', delimiter=';')

# Drop unnamed columns if present (from trailing delimiters)
air_quality_data = air_quality_data.loc[:, ~air_quality_data.columns.str.contains('^Unnamed')]

# Drop rows with missing target values
air_quality_data = air_quality_data.dropna(subset=['CO(GT)'])

# Data Preprocessing
# Use 'CO(GT)' as the target column (replace with another if needed)
X = air_quality_data.drop(columns=['CO(GT)'])
y = air_quality_data['CO(GT)']

# Fill missing values for features
X = X.fillna(X.median(numeric_only=True))

# Drop non-numeric columns before scaling
X = X.select_dtypes(include=['number'])

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Training the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# 5. Naive Submission 2

We can now make another submission based on the refined pipeline.

# 6. EDA and Data Preprocessing

Exploration of the dataset to prepare it for modeling.

In [ ]:

import matplotlib.pyplot as plt

# EDA - Basic statistics and visualizations
air_quality_data.describe()

# Plotting a histogram for distribution of a column
plt.hist(air_quality_data['CO(GT)'], bins=20)
plt.title('CO(GT) Distribution')
plt.xlabel('CO(GT)')
plt.ylabel('Frequency')
plt.show()


# 7. Model Training

Train the model with the data and process.

In [ ]:

# Example using RandomForest, but can be extended to other models as well
model.fit(X_train_scaled, y_train)


# 8. Model Evaluation

Evaluating the model's performance.

## 8.1 Model Evaluation using Accuracy Score

In [ ]:

from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


## 8.2 Model Evaluation using Confusion Matrix

In [ ]:

from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


# 9. Decision Tree with Gridsearch

Implementing Decision Tree and tuning hyperparameters with GridSearchCV.

In [ ]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Define the model and parameters for GridSearch
dt = DecisionTreeClassifier(random_state=42)
param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)

print(f'Best parameters: {grid_search.best_params_}')


# 6. EDA and Data Preprocessing

Exploring the data to identify key patterns, visualizations, and outliers.

In [ ]:

import seaborn as sns

# Convert 'CO(GT)' to numeric (replace comma with dot and coerce errors)
air_quality_data['CO(GT)_num'] = pd.to_numeric(air_quality_data['CO(GT)'].str.replace(',', '.'), errors='coerce')

# Pairplot to visualize the relationships between features
sns.pairplot(air_quality_data.select_dtypes(include=['float64']))
plt.show()

# Boxplot for detecting outliers in the 'CO(GT)' column
plt.figure(figsize=(8, 6))
sns.boxplot(x=air_quality_data['CO(GT)_num'])
plt.title('Boxplot for CO(GT)')
plt.show()


### Outlier Detection

In [ ]:

# Detecting outliers using the IQR method
# Convert 'CO(GT)' to numeric if not already
co_numeric = pd.to_numeric(air_quality_data['CO(GT)'], errors='coerce')
Q1 = co_numeric.quantile(0.25)
Q3 = co_numeric.quantile(0.75)
IQR = Q3 - Q1

# Identifying outliers
outliers = air_quality_data[(co_numeric < (Q1 - 1.5 * IQR)) | (co_numeric > (Q3 + 1.5 * IQR))]
print(f'Number of outliers: {outliers.shape[0]}')
sns.boxplot(x=co_numeric)
plt.title('Boxplot with Outliers Detected')
plt.show()


# 8. Model Evaluation

We will compare model performances with various metrics.

### 8.1 Model Evaluation using Accuracy, Precision, Recall, and F1-Score

In [ ]:

from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the model with accuracy, precision, recall, and f1-score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Displaying the evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')


### 8.2 ROC Curve

In [ ]:

from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np

# Binarize the output
classes = np.unique(y_test)
y_test_bin = label_binarize(y_test, classes=classes)
y_score = model.predict_proba(X_test_scaled)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(classes)):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve for each class
plt.figure(figsize=(8, 6))
for i in range(len(classes)):
    plt.plot(fpr[i], tpr[i], lw=2, label=f'Class {classes[i]} (AUC = {roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve (Multiclass)')
plt.legend(loc='lower right')
plt.show()


# 9. Decision Tree with Gridsearch

We will implement Decision Tree and tune its hyperparameters.

In [ ]:

# Example of hyperparameter tuning using GridSearchCV for Decision Tree
dt = DecisionTreeClassifier(random_state=42)
param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)

# Best parameters from GridSearch
print(f'Best parameters from GridSearch: {grid_search.best_params_}')